<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Objectif :
* interfacer un programme **prog** avec jupyter pour qu'il puisse avoir des entrées/sorties dans jupyter (seul python sait bien avoir des entrées/sorties au départ)
* définir un lanceur/compagnon python du programme **prog** qui gère les entrées/sorties :
```python
def lanceur(prog):
    os.system( prog+ " < fournir entrée par fifo + eventuellement recup sorties &")
    #"lancer le fournisseur d'entree (en python) vers fifo"
```

Les soucis et les solutions : 
* le programme **prog**s'exécute avec jupyter dans un shell (invisible), il n'a pas moyen de lui donner des entrées (et les sorties ne s'affichent pas toujours tout de suite) : prévoir un compagnon "python" qui fournira des entrées (en python, input fonctionne) via une fifo
* les fifo se bloquent d'un côté ou l'autre s'il n'y a personne de l'autre côté : il faut lancer le processus du programme **prog** en tâche de fond avant le début des saisies/affichages
* si le programme **prog** doit avoir des entrées et des sorties, les blocages peuvent arriver des 2 côtés : éviter des fifo simultanément en entrée et en sortie 
* comme le programme **prog** est en tâche de fond, utiliser par ex. la sortie erreur pour que l'affichage se fasse en jupyter et sans bloccage (il semble que la sortie erreur soit mieux gérée que la sortie std sous jupyter, en tout cas pour les processus en tâche de fond lancés dans un shell (invisible ?), en tout cas, la sortie erreur apparait sur jupyter et immédiatement, même pour un processus en tâche de fond lancés dans un shell (invisible ?)) : en C, par exemple, passer par warnx (sinon il y a d'autres solutions en utilisant des fichiers temporaires en les ouvrant et les fermant explicitement souvent et en les oscultant régulièrement)
* si plusieurs saisies arrivent sur la fifo, chacune risque de déclencher **prog** et fermer la fifo (dommage) : prévoir de garder la fifo ouverte un certains temps
* au bout du délai, enlever la fifo (pour que les suivants ne récupère pas des données non utilisées)

Fichiers :
* lanceur/compagnon (en entrée : le programme, une longueur de boucle de saisie, ici 4 par défaut, et un temps d'exécution, ici 20s)

In [18]:
import os
import time

def lanceur(prog,nb=4,tps=20):
    os.system("if [ -p monPipeEnEntree ]; then rm monPipeEnEntree; fi")
    os.system("mkfifo monPipeEnEntree")
    os.system("sleep "+str(tps)+" > monPipeEnEntree &")         # important : ouvrir et garder le pipe ouvert pour un temps assez grand
    os.system("(sleep "+str(tps+1)+" ; rm monPipeEnEntree) &")
    os.system(prog+" < monPipeEnEntree &")                      # important : lancer le programme (prévoir que les sorties soient visibles, par ex. sur la sortie erreur)
    for i in range(nb):
        time.sleep(0.5)
        x=input("?")                                            # important : faire une saisie (cela peut servir)
        os.system("echo "+x+" > monPipeEnEntree")               # important : envoyer la saisie sur le programme (si cela peut servir)

Le programme prog :

In [13]:
%%writefile prog.c
#include <stdio.h>
#include <stdint.h>
#include <err.h>

int main(int argc, char * argv[]) {
int32_t z;
for(int i=0;i<4;i++) {
  warnx("Donner un entier, svp.");
  scanf("%d",&z);
  warnx("%d =>  %d",z,(int16_t) z);}
return 0;}

Overwriting prog.c


In [14]:
%sx gcc -o prog.e prog.c

[]

In [17]:
lanceur("./prog.e",4,30)

prog.e: Donner un entier, svp.


?4


prog.e: 4 =>  4
prog.e: Donner un entier, svp.


?6


prog.e: 6 =>  6
prog.e: Donner un entier, svp.


?8


prog.e: 8 =>  8
prog.e: Donner un entier, svp.


?95


prog.e: 95 =>  95
